# 3. Neural Network Classification


## 3.1 Creating dataset


In [ ]:
import torch as tc
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
import pandas as pd

In [ ]:
# Make a 1000 circles
n_samples = 1000

# create a circle
X, y = make_circles(n_samples, noise=0.03, random_state=42)
print(len(X), len(y))
print(f"Firts 5 samples of X:\n {X[:5]}")
print(f"Firts 5 samples of y:\n {y[:5]}")

In [ ]:
# make a df
circles = pd.DataFrame({"X1": X[:, 0], "X2": X[:, 1], "Label": y})
circles.head(10)

In [ ]:
plt.scatter(x=X[:, 0], y=X[:, 1], c=y, cmap="viridis")

## 3.2 Check input and output shape


In [ ]:
X.shape, y.shape

In [ ]:
# Viewing features and labels
X_sample = X[0]
y_sample = y[0]

print(f"Values for one sample of X:\n{X_sample}\nAnd the same for y:\n{y_sample}")
print(
    f"Shapes for one sample of X:\n{X_sample.shape}\nAnd the same for y:\n{y_sample.shape}"
)

## 3.3 Turning Data into tensors


In [ ]:
X = tc.from_numpy(X).type(tc.float)
y = tc.from_numpy(y).type(tc.float)

X[:5], y[:5]

In [ ]:
# split data into train test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.3
)
print(X_test[:5], X_train[:5], y_train[:5], y_test[:5])

## 3.4 Building a model


In [ ]:
# 1. Custruct a model
class CircleModelV1(nn.Module):
    def __init__(self):
        super().__init__()
        # 2. Create 2 nn.linear layer to handle shapes of our data
        # self.layer1 = nn.Linear(in_features=2, out_features=5)
        # self.layer2 = nn.Linear(in_features=5, out_features=1)

        self.two_linear_layers = nn.Sequential(
            nn.Linear(in_features=2, out_features=5),
            nn.Linear(in_features=5, out_features=1),
        )

    # 3. Define a forward method that outlines the forward pass
    def forward(self, x):
        # return self.layer2(self.layer1(x))
        return self.two_linear_layers(x)


# 4. Instantiate an instance of our model class and send it to target device
model0 = CircleModelV1().to(device="cpu")
model0

In [ ]:
# making some predictions
# model0.state_dict()
with tc.inference_mode():
    preds1 = model0(X_test)
print(f"Length of predictions: {len(preds1)},\nShape: {preds1.shape}")
print(f"Length of test samples: {len(X_test)},\nShape: {X_test.shape}")
print(f"Length of train samples: {len(X_test)},\nShape: {X_train.shape}")
print(f"First 10 predictions:\n{preds1[:10]}")

## 3.5 Setting up a loss function


In [ ]:
loss_fn = nn.BCEWithLogitsLoss  # sigmoid activation function
optimiser = tc.optim.SGD(params=model0.parameters(), lr=0.1)

# model0.state_dict()

In [ ]:
# evaluation matrix
def accuracy_fc(y_true, y_pred):
    correct = tc.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

## 3.6 Trainig Model


In [ ]:
# 0 View the first 5 outputs of the forward pass loss data
# 1. Forward Pass
# 2. Calculate the loss
# 3. Optimize the zero grad
# 4. Back propagation
# 5. Optimise the steps

In [ ]:
# 0.
model0.eval()
with tc.inference_mode():
    y_logits = model0(X_test)[:5]
y_logits

In [ ]:
# using the sigmoid activation function
y_preds_probs = tc.sigmoid(y_logits)
tc.round(y_preds_probs)

In [ ]:
# Find the predicted probabilities
y_preds = tc.round(y_preds_probs)
# in full
y_preds_labels = tc.round(tc.sigmoid(model0(X_test)[:5]))
# check for equality
print(tc.eq(y_preds.squeeze(), y_preds_labels.squeeze()))
# getting rid of extra dimensions
y_preds.squeeze()

### 3.6.1 Building a training and test loop


In [ ]:
tc.manual_seed(42)
# setting the number of epochs
epochs = 100

for epoch in range(epochs):
    # training
    model0.train()
    # 1.
    y_logits = model0(X_train).squeeze()
    y_pred = tc.round(tc.sigmoid(y_logits))

    # 2
    loss = loss_fn(y_logits, y_train)
    acc = accuracy_fc(y_true=y_train, y_pred=y_pred)
    
    # 3
    optimiser.zero_grad()
    
    # 4
    loss.backward()
    
    # 5
    optimiser.step()
    model0.eval()
    with tc.inference_mode():
      # 1
      test_logits = model0(X_test).squeeze()
      test_pred = tc.round(tc.sigmoid(test_logits))
      
      # 2
      test_loss = loss_fn(test_logits,y_test)
      test_acc = accuracy_fc(y_true=y_test,y_pred=test_pred)
      
    # print what happenned
    if epoch % 10 == 0:
      print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

## 3.7 Refining our model

In [ ]:
class CircleModelV2(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1 = nn.Linear(in_features=2,out_features=10)
    self.layer_2 = nn.Linear(in_features=10,out_features=10)
    self.layer_3 = nn.Linear(in_features=10,out_features=1)
    
  def forward(self,x):
    return self.layer_3(self.layer_2(self.layer_1(x)))
  
model1 = CircleModelV2()
model1

In [ ]:
# setting up a loss function
loss_fn1 = nn.BCEWithLogitsLoss()

# creating optimiser
optimizer = tc.optim.SGD(params=model1.parameters(),lr=0.1)

In [ ]:
# training and evaluating loop model
tc.manual_seed(42)

epochses = 1000

for epochse in range(epochses):
    # training
    model1.train()
    # 1. forward pass
    y_logits1 = model1(X_train).squeeze()
    y_pred1 = tc.round(tc.sigmoid(y_logits1))
    # calculate the loss/acc
    loss1 = loss_fn1(y_logits1, y_train)
    acc = accuracy_fc(y_true=y_train, y_pred=y_pred1)
    # 3. Optimize zero gradient
    optimizer.zero_grad()
    # 4. loss backwards
    loss1.backward()
    # 5. optimiser step
    optimizer.step()

    # testing
    model1.eval()
    with tc.inference_mode():
        # 1. forward pass
        test_logits1 = model1(X_test).squeeze()
        test_pred1 = tc.round(tc.sigmoid(test_logits1))
        # 2. calculate the loss
        test_loss1 = loss_fn1(test_logits1, y_test)
        test_acc1 = accuracy_fc(y_true=y_test, y_pred=test_pred1)
        # print what happenned
        if epochse % 50 == 0:
            print(
                f"Epoch: {epochse} | Loss: {loss1:.5f}, Acc: {acc:.2f}% | Test loss: {test_loss1:.5f}, Test acc: {test_acc1:.2f}%"
            )